In [1]:
import pandas as pd
df = pd.read_csv ("data83.csv")
print(df)
# print(len(X_train_o[0,:])) #经过多项式变化的特征数，和前三项不能同时存在

     Sample  Z_1  Z_2    Z_    Z+  Nmen_1  Nmen_2  Nmen_  Nmen+   X_1  ...  \
0     Ni_Sc   28   21   3.5  24.5      61      11   25.0   36.0  1.91  ...   
1     Ni_Mn   28   25   1.5  26.5      61      52    4.5   56.5  1.91  ...   
2     Ni_Co   28   27   0.5  27.5      61      58    1.5   59.5  1.91  ...   
3     Ni_Cu   28   29   0.5  28.5      61      64    1.5   62.5  1.91  ...   
4     Ni_Nb   28   41   6.5  34.5      61      47    7.0   54.0  1.91  ...   
..      ...  ...  ...   ...   ...     ...     ...    ...    ...   ...  ...   
161   Os_WE   74   76   1.0  75.0      51      57    3.0   54.0  2.36  ...   
162   Re_WE   74   75   0.5  74.5      51      54    1.5   52.5  2.36  ...   
163  Mo_TiE   22   42  10.0  32.0      43      50    3.5   46.5  1.54  ...   
164  Ag_PdE   46   47   0.5  46.5      62      65    1.5   63.5  2.20  ...   
165  Cu_ScE   21   29   4.0  25.0      11      64   26.5   37.5  1.36  ...   

        R+  Rp_1  Rp_2    Rp_    Rp+  Rv_1  Rv_2    Rv_    Rv+ 

In [2]:
import numpy as np
import pandas as pd
import csv
import pickle
from joblib import dump, load
from sklearn import utils
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from gplearn.genetic import SymbolicTransformer, SymbolicRegressor
# SymbolicRegressor是拟合公式使用的
import datetime
import time

In [3]:
def scaling(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    return x

In [4]:
#read data
rawData = pd.read_excel('data83.xlsx')
# 将原始数据分割为两部分

inputData = np.array(rawData.iloc[:,1:-1])
inputData = scaling(inputData)
outputData = np.array(rawData.iloc[:,-1])
outputData

array([1.72204686, 2.48023511, 2.72016285, 2.42275538, 3.00608784,
       2.84794849, 1.66776322, 2.10723258, 2.36733483, 3.52161669,
       1.09030616, 2.36212358, 2.0245006 , 3.35408932, 2.6980144 ,
       3.54973412, 3.37608413, 2.16533815, 3.40483396, 3.39201234,
       3.82601077, 3.31779236, 2.06239574, 1.64833419, 3.06490299,
       2.74904436, 3.33422146, 3.0111769 , 2.07869049, 2.85304536,
       3.44504737, 3.4176552 , 3.0080437 , 3.39242458, 2.42228493,
       2.93838485, 2.31950789, 2.48223148, 3.14153579, 3.28641994,
       2.90627959, 2.47834665, 2.00530465, 2.68309905, 2.42075079,
       2.25066826, 1.95331541, 3.60861019, 2.58489195, 2.94485963,
       2.40919182, 3.30277117, 4.0522839 , 3.04588146, 3.54106302,
       2.16542693, 1.99117679, 2.42687993, 3.37497717, 2.96447816,
       1.91755539, 1.49116469, 2.4261312 , 3.28290773, 3.43251198,
       2.25613498, 2.35834864, 3.2429655 , 1.37212891, 1.77518725,
       3.1687204 , 3.18931332, 2.98921794, 2.73397299, 3.06010

In [5]:
best_fitness = float('-inf')
best_pc = 0
best_pcoef = 0
best_ps = 0
best_features = []
# 对最佳特征进行定义
best_gp = []
best_mean = float('inf')
function_set = ['add', 'sub', 'mul', 'div', 'abs', 'sqrt']
pc = np.arange(0.1, 0.51, 0.1)
pcoef =  np.arange(0.001, 0.006, 0.001)
pc_grid, pcoef_grid = np.meshgrid(pc, pcoef)
pc_grid = np.array(pc_grid).flatten()
pcoef_grid = np.array(pcoef_grid).flatten()
dimNum = len(pc_grid)
count = 0
    

In [6]:
for i in range(dimNum):# 对网格参数的搜索
    pc0 = pc_grid[i]
    pcoef0 = pcoef_grid[i]
    ps_max = 1-pc0; ps_min = 0.92-pc0
    ps = np.arange(ps_min, ps_max, 0.01)
    ps = ps/3
    dimNum_ps = len(ps)
    for j in range(dimNum_ps):# 对网格参数的定义
        ps0 = ps[j]
        ph0 = ps0
        pp0 = 0.999999999-pc0-ph0-ps0
        gp = SymbolicTransformer(population_size=5000,  function_set=function_set, generations=20, n_components=6,
                        p_crossover=pc0, p_subtree_mutation=ps0,parsimony_coefficient=pcoef0, p_hoist_mutation=ph0, p_point_mutation=pp0,
                         const_range=(-1.0, 1.0), metric='pearson',tournament_size=20, verbose=1)
        print('********************************Round %d********************************' %(count+1))
        print('----------The round of pc=%.6f ps=%.6f pcoef=%.6f pp=%.6f ph=%.6f-----------'
              %(pc0, ps0, pcoef0, pp0, ph0))
        gp.fit(inputData, outputData)
        fitness = gp.run_details_['best_fitness'][-1]
        print('The round of fitness=%.6f\n'%fitness)
        features_pd = pd.DataFrame(best_features)
        #PCC_GH_1 = abs(features_pd.corr('spearman').iloc[0,1:])
        #PCC_GH_2 = abs(features_pd.corr('spearman').iloc[1,[0,2,3,4]])
        #PCC_GH_3 = abs(features_pd.corr('spearman').iloc[2,[0,1,3,4]])
        #PCC_GH_4 = abs(features_pd.corr('spearman').iloc[3,[0,1,2,4]])
        #PCC_GH_5 = abs(features_pd.corr('spearman').iloc[4,[0,1,2,3]])
        #PCC_GH_mean = np.mean(PCC_GH_1)+np.mean(PCC_GH_2)+np.mean(PCC_GH_3)+np.mean(PCC_GH_4)+np.mean(PCC_GH_5)
        #PCC_GH_mean
        count = count+1
        if fitness > best_fitness :
            best_pc = pc0
            best_pcoef = pcoef0
            best_ps = ps0
            best_features = gp.transform(inputData)
            best_fitness = fitness
            best_gp = gp      
    

********************************Round 1********************************
----------The round of pc=0.100000 ps=0.273333 pcoef=0.001000 pp=0.353333 ph=0.273333-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.32         0.135625        5         0.588801              N/A     26.95s
   1     4.88          0.32091        7         0.666743              N/A     35.34s
   2     3.54         0.413819        7         0.709406              N/A     50.38s
   3     4.72         0.442767        9         0.720565              N/A     47.23s
   4     6.19          0.48619        9         0.720565              N/A     45.82s
   5     6.51         0.521824        9         0.720565              N/A     42.49s
   6     7.18         0.547368       16         0.744151              N/A   

  16    15.99         0.647408       29         0.801169              N/A     11.06s
  17    15.81         0.644841       34           0.8071              N/A      6.97s
  18    17.87         0.657252       30         0.812308              N/A      3.71s
  19    18.29           0.6605       30         0.812308              N/A      0.00s
The round of fitness=0.812308

********************************Round 5********************************
----------The round of pc=0.100000 ps=0.286667 pcoef=0.001000 pp=0.326667 ph=0.286667-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.42         0.138572        6         0.605743              N/A      1.06m
   1     5.31         0.322521        7         0.664378              N/A     58.24s
   2     3.89         0.413299        5      

  12     9.70         0.572517       18         0.775968              N/A     24.19s
  13    10.45         0.584211       18         0.775968              N/A     20.64s
  14    12.83          0.60383       18         0.778978              N/A     17.36s
  15    13.37         0.622132       25         0.786819              N/A     14.57s
  16    13.38         0.626004       25         0.786819              N/A     10.56s
  17    15.44         0.643687       27          0.79265              N/A      7.04s
  18    16.53         0.649636       30         0.804662              N/A      3.60s
  19    17.21         0.647312       30         0.804662              N/A      0.00s
The round of fitness=0.804662

********************************Round 9********************************
----------The round of pc=0.200000 ps=0.240000 pcoef=0.001000 pp=0.320000 ph=0.240000-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- --------------

   8    18.87         0.650528       27         0.817636              N/A     41.74s
   9    18.50         0.648337       56         0.824375              N/A     39.66s
  10    18.18         0.651035       59         0.843328              N/A     32.11s
  11    24.04         0.687461       66          0.85302              N/A     29.43s
  12    26.48         0.703472       67          0.85506              N/A     25.48s
  13    28.44         0.707991       60           0.8565              N/A     22.12s
  14    31.25         0.716804       75         0.858699              N/A     19.46s
  15    32.20          0.72394       53         0.866845              N/A     15.31s
  16    31.68         0.727763       53         0.866845              N/A     11.52s
  17    31.16         0.723851       52         0.866845              N/A      7.66s
  18    30.26         0.729105       52         0.866845              N/A      3.74s
  19    28.73         0.727506       60         0.871655         

   4     5.99         0.499736       16         0.719376              N/A     50.89s
   5     5.88         0.525154        6         0.719741              N/A     46.29s
   6     5.81         0.534245        7         0.720437              N/A     43.30s
   7     5.34         0.537125        7         0.720437              N/A     40.37s
   8     5.71         0.538451        7         0.720437              N/A     37.38s
   9     5.85          0.54453        7         0.720437              N/A     33.03s
  10     5.97         0.535441        7         0.720437              N/A     31.24s
  11     5.77         0.537284        7         0.720437              N/A     26.17s
  12     5.81         0.537162        7         0.720437              N/A     23.27s
  13     5.85         0.533366        7         0.720437              N/A     20.56s
  14     5.84          0.53938        7         0.720437              N/A     17.08s
  15     5.79         0.533335        7         0.720437         

...

   0    13.19         0.135877        6         0.611328              N/A      1.08m
   1     3.86         0.329117        7         0.719741              N/A     58.25s
   2     3.26         0.426413        7         0.719741              N/A     53.70s
   3     4.42         0.482856        8         0.720437              N/A     53.90s
   4     6.04         0.520398       13         0.733892              N/A     48.94s
   5     5.56         0.538808        8         0.720437              N/A     47.18s
   6     5.35         0.538326        7         0.720437              N/A     44.64s
   7     5.84         0.552326        7         0.720437              N/A     39.97s
   8     5.87          0.55125        8         0.722711              N/A     36.11s
   9     5.85         0.550539        7         0.720437              N/A     34.07s
  10     5.97         0.555486        7         0.720437              N/A     30.35s
  11     5.85         0.555616        7         0.720437         

   0    12.89         0.134027        3         0.605152              N/A      1.07m
   1     4.06         0.326942        7         0.636246              N/A     58.53s
   2     3.05         0.440894        8          0.64449              N/A     53.68s
   3     3.64         0.486034        6         0.695616              N/A     52.78s
   4     4.95         0.516629        9         0.712388              N/A     49.24s
   5     6.23           0.5412        9         0.730491              N/A     45.74s
   6     6.84         0.551617       10         0.746013              N/A     44.11s
   7     6.54         0.546809       18         0.755214              N/A     39.41s
   8     6.21         0.558658       17         0.778495              N/A     36.63s
   9     7.65         0.561822       20         0.790084              N/A     33.92s
  10     9.90         0.604087       20         0.806727              N/A     30.68s
  11    10.51         0.608747       20         0.806727         

   0    13.22         0.136779       13         0.598326              N/A      1.05m
   1     3.99         0.327353        5         0.666743              N/A     57.14s
   2     3.29         0.429209        5         0.709807              N/A     55.27s
   3     4.76         0.446914       10         0.714546              N/A     51.53s
   4     6.23         0.485875       10         0.729092              N/A     48.31s
   5     5.99         0.523955       20         0.749508              N/A     47.07s
   6     5.41         0.549383       11         0.744483              N/A     42.68s
   7     5.72         0.555302       11         0.744483              N/A     39.21s
   8     5.88          0.55883        7         0.720437              N/A     38.00s
   9     5.89         0.556273        7         0.720437              N/A     32.99s
  10     5.88         0.566931        7         0.720437              N/A     29.55s
  11     5.94         0.558619        7         0.720437         

   0    13.32         0.137784        3         0.605152              N/A      1.06m
   1     3.84         0.335226        7         0.709807              N/A     57.40s
   2     2.96         0.454461        6         0.709807              N/A     53.69s
   3     3.92         0.501575        5         0.709807              N/A     52.03s
   4     5.80         0.543956       11         0.725474              N/A     48.56s
   5     5.76         0.566655       15         0.740735              N/A     45.09s
   6     5.29         0.567332        7         0.720437              N/A     43.89s
   7     5.78         0.576323        7         0.720437              N/A     39.21s
   8     5.93          0.57913        7         0.720437              N/A     35.94s
   9     5.92         0.578046        7         0.720437              N/A     32.68s
  10     5.87         0.579254        7         0.720437              N/A     30.24s
  11     5.92         0.581026        7         0.720437         

   0    12.93         0.137088       12         0.562904              N/A      1.05m
   1     3.93         0.333821        6         0.638883              N/A     57.42s
   2     2.83         0.443165        6         0.695616              N/A     55.26s
   3     3.77         0.463301        8         0.720765              N/A     51.28s
   4     5.46         0.516481        9         0.728948              N/A     48.77s
   5     7.01         0.551756       13         0.767062              N/A     49.23s
   6     7.61         0.571448       16         0.791043              N/A     52.87s
   7     9.31         0.595325       19         0.800679              N/A     40.71s
   8    11.37         0.632469       25         0.826237              N/A     38.70s
   9    11.64         0.637171       25         0.826237              N/A     33.45s
  10    13.09          0.65226       19         0.826499              N/A     30.39s
  11    14.57         0.665099       19         0.826499         

In [7]:
#output result
print('The best fitness=%.3f;The best pc=%.6f;The best pcoef=%.6f;The best ps=%.6f'%(best_fitness, best_pc, best_pcoef, best_ps))
print('The best features: %s'%best_features)
print('The best formula: %s'%best_gp)

The best fitness=0.900;The best pc=0.500000;The best pcoef=0.001000;The best ps=0.146667
The best features: [[15.80133665  3.34980992  3.73089174 16.03692394  4.79373674  5.14346086]
 [ 6.92465249  1.95128381  2.30999124  4.13921746  2.63459338  3.12462451]
 [ 8.47859411  1.61439405  2.12816807  4.22695235  2.65747343  3.31457449]
 [11.88410327  2.71514147  3.1544054  10.83344979  4.05661361  4.21440038]
 [ 4.74515162  1.76747534  1.82086554  4.64959195  2.69761148  2.56864309]
 [ 7.14206465  2.01227598  2.40168945  5.02789926  2.77679479  3.0268498 ]
 [17.54623242  3.32916502  3.71188849 15.09419965  4.84323129  5.11888127]
 [11.42303068  2.86335985  2.91289251 11.48589506  4.20020324  4.03873869]
 [11.03632964  2.56906824  2.92161541 10.16931334  4.00633457  4.10165984]
 [ 1.11022922  0.42382852  0.46917432  1.87915818  1.21066224  1.02388845]
 [19.69258679  4.27997639  4.20751743 20.10341911  5.88963512  5.98935489]
 [10.76148112  2.67408153  2.78637354 10.03127531  3.77615001  4.02

In [8]:
#saving model and data
path = "_6D"
data = pd.DataFrame(best_features)
data.to_csv("list_Gpfeature"+path+".csv",index=False,sep=',')
demension = path.split('_')[1]
pickle.dump(best_gp, open(demension+"gp"+path+".pickle.dat", "wb"))
data = pd.DataFrame(best_features)
data.to_csv(demension+"list_Gpfeature"+path+".csv",index=False,sep=',')
